In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical 

import keras
from keras_tuner import HyperModel, RandomSearch
from keras import layers, Sequential
from keras import regularizers
from keras.callbacks import EarlyStopping
from keras.models import load_model

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

import matplotlib.pyplot as plt

import seaborn as sns

    Oversampling

In [2]:
def oversampling(X_train, y_train):

    smote = SMOTE(random_state=42)
    X_train_os, y_train_os = smote.fit_resample(X_train, y_train)

    return X_train_os, y_train_os

    Undersampling

In [3]:
def undersampling(X_train,y_train):

    rus = RandomUnderSampler(random_state=42)
    X_train_us, y_train_us = rus.fit_resample(X_train, y_train)

    return X_train_us, y_train_us

    Logistic Regression

In [4]:
def logistic_regression(X_train, y_train, X, WC):
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X = scaler.transform(X)
    
    if WC:
        model = LogisticRegression(class_weight='balanced', max_iter=1000)
    else:
        model = LogisticRegression(max_iter=1000)
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X)
    
    return y_pred

    SVM

In [5]:
# Função para SVM com weighted classes
def svm_classification(X_train, y_train, X, WC):
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X = scaler.transform(X)

    # Define o modelo com ou sem weighted classes
    if WC:
        model = SVC(class_weight='balanced', kernel='linear')  # Podes experimentar outros kernels
    else:
        model = SVC(kernel='linear')

    model.fit(X_train, y_train)
    y_pred = model.predict(X)
    return y_pred

    Model evaluation

In [6]:
def evaluate_model(cm):   
    tn = cm[0, 0]  # True negatives
    fp = cm[0, 1]  # False positives
    fn = cm[1, 0]  # False negatives
    tp = cm[1, 1]  # true positives

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (tp + tn) / (tp + fp + fn + tn)
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    sensitivity = recall  # ou: tp / (tp + fn)
    balanced_accuracy = (specificity + sensitivity) / 2

    print(f"f1 score:{f1_score}")
    print(f"accuracy:{accuracy}")
    print(f"balanced_accuracy:{balanced_accuracy}")

    Confusion Matrix

In [ ]:
X_train = np.load("Xtrain1.npy")  
Y_train = np.load("Ytrain1.npy")  

X_train = X_train.astype('float32') / 255.0

X_train, X_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

print(X_train.shape)
print(y_train.shape)

WC = False  # Weighted classes
OS = False # Oversampling
US = False # Undersampling

LR = True  # Logistic Regression
SVM = False # SVM

if OS:
    print("oversampling")
    X_train, y_train = oversampling(X_train, y_train)
elif US:
    print("undersampling")
    X_train, y_train = undersampling(X_train,y_train)
elif WC:
    print("Class Weighted")
else:
    print("Imbalanced")

if LR:
    print("LOGISTIC REGRESSION")
    binary_predictions = logistic_regression(X_train, y_train, X_val, WC)

if SVM:
    print("SVM")
    binary_predictions = svm_classification(X_train, y_train, X_val, WC)

cm = confusion_matrix(y_val, binary_predictions)

if cm is not None:
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=np.unique(Y_train), 
                yticklabels=np.unique(Y_train))
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.title('Confusion Matrix')
    plt.show()

    print("Confusion Matrix:")
    print(cm)
    evaluate_model(cm)
else:
    print("A matriz de confusão não foi criada. Verifique as condições.")